In [ ]:
include("slim_reader.jl")
include("Network.jl")
include("Encoder.jl")

using Knet, ImageMagick, Colors, Images, Sloth, DRAW, Random
BATCH_SIZE = 50
EPOCH = 5
NUMBATCHES = 200000
slim_data_dir = "/home/ec2-user/efs/slim-dataset"
syn_train_files = Slim_Reader.get_train_filenames(slim_data_dir, true, false)
# Construct training dataset object for synthetic data
syn_train_data = Slim_Reader.make_dataset_object(syn_train_files, BATCH_SIZE)
syn_train_batch = Slim_Reader.get_next_batch(syn_train_data, KnetArray{Float32}, false, true);
images, captions, cameras = getdata(syn_train_batch)

In [ ]:
goldimages = creategoldimg(images)

In [ ]:
n = Network()

In [ ]:
function loss(n::Network, images, captions, cameras, final_output_gold)
    output = n(images, captions, cameras)
    recon = bce(output.rimages, final_output_gold) #*32 *32 # neden 32*32 ile carptik
    mu_2 = DRAW.square(output.mu)
    kl = 0.5 * sum(output.sigma + mu_2 .- 1 - output.logsigma)
    return recon + kl
end

In [ ]:
function update_weights!(n::Network, images, captions, cameras, y)
    J = @diff loss(n, images, captions, cameras, y)
    for par in params(n)
        g = grad(J, par)
        update!(value(par), g; lr=0.1)
    end
    return value(J)
end

In [ ]:
function epoch!(n::Network, syn_train_data) # last loss = loss before last update
    loss_values = []
    for i in 1:EPOCH
        loss = 0
        for j in 1:NUMBATCHES
            syn_train_batch = Slim_Reader.get_next_batch(syn_train_data, KnetArray{Float32}, false, true)
            images, captions, cameras = getdata(syn_train_batch)
            y = creategoldimg(images)
            loss = loss + update_weights!(n, images, captions, cameras, y)
        end
        push!(loss_values, loss/NUMBATCHES)
        Knet.save(string("trained_model", i, ".jld2"), "results", n)
    end
    return loss_values
end

In [ ]:
loss_values = epoch!(n, syn_train_data)